# Recommender System Modeling
In this notebook, we will work through multiple iterations of our movie recommender system model. First, we will create a simpler model using just the movie reviews from IMDb. This will give us a baseline model that we can potentially improve on by adding more detailed features.

In [12]:
# Imports
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity

## Simple Recommender System
This is a simple recommender system using cosine similarity that can be improved upon and replaced when all our data is properly formatted.

In [6]:
# Let's try this with only our MovieLens reviews first
ml_reviews = pd.read_csv('../Data/Large-Data/ml_reviews.csv')
ml_reviews

,user_id,imdb_id,scaled_rating,title
0,1,tt0110912,10.0,Pulp Fiction
1,1,tt0111495,7.0,Trois couleurs: Rouge
2,1,tt0108394,10.0,Trois couleurs: Bleu
3,1,tt0114787,10.0,Underground
4,1,tt0045152,7.0,Singin' in the Rain
...,...,...,...,...
24969860,162541,tt0382932,9.0,Ratatouille
24969861,162541,tt0389790,5.0,Bee Movie
24969862,162541,tt0952640,4.0,Alvin and the Chipmunks
24969863,162541,tt0468569,8.0,The Dark Knight


In [4]:
# First let's create a pivot table with a small sample of movie reviews
sampled_reviews = ml_reviews.sample(frac=0.3, random_state=42)

In [7]:
sampled_reviews = ml_reviews[ml_reviews['user_id']<81000]
sampled_reviews

,user_id,imdb_id,scaled_rating,title
0,1,tt0110912,10.0,Pulp Fiction
1,1,tt0111495,7.0,Trois couleurs: Rouge
2,1,tt0108394,10.0,Trois couleurs: Bleu
3,1,tt0114787,10.0,Underground
4,1,tt0045152,7.0,Singin' in the Rain
...,...,...,...,...
12511161,80999,tt5657846,9.0,Daddy's Home 2
12511162,80999,tt5095030,6.0,Ant-Man and the Wasp
12511163,80999,tt6921996,9.0,Johnny English Strikes Again
12511164,80999,tt1727824,10.0,Bohemian Rhapsody


In [8]:
print(f"Reviews: {len(sampled_reviews['imdb_id'])}")
print(f"Movies: {sampled_reviews['imdb_id'].nunique()}")
print(f"Users: {sampled_reviews['user_id'].nunique()}")

Reviews: 12511166
Movies: 53786
Users: 80999


In [23]:
sampled_reviews['title'].nunique()

13868

In [40]:
print(f"Reviews: {len(sampled_reviews['imdb_id'])}")
print(f"Movies: {sampled_reviews['imdb_id'].nunique()}")
print(f"Users: {sampled_reviews['user_id'].nunique()}")

Reviews: 7490960
Movies: 44165
Users: 162532


In [4]:
pivot = sampled_reviews.pivot_table(values='scaled_rating', index='title', columns='user_id')
pivot.head()

user_id,1,2,3,4,5,8,9,10,12,13,...,162528,162529,162532,162533,162534,162535,162536,162538,162539,162540
title,,,,,,,,,,,,,,,,,,,,,
#Followme,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#Nerealnaya lyubov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
$5 a Day,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
$9.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
$ellebrity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
pivot.loc['#Followme']

user_id
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
          ..
162535   NaN
162536   NaN
162538   NaN
162539   NaN
162540   NaN
Name: #Followme, Length: 89946, dtype: float64

In [ ]:
#pivot = ml_reviews.pivot_table(values = 'scaled_rating', index = 'title', columns = 'user_id')

#pivot.head()

In [5]:
pivot.shape

(13868, 89946)

In [6]:
pivot_sparse = sparse.csr_matrix(pivot.fillna(0))

In [7]:
distances = pairwise_distances(pivot_sparse, metric = 'cosine')

In [9]:
similarities = 1.0 - distances

In [10]:
recommender = pd.DataFrame(similarities, index = pivot.index, columns=pivot.index)

recommender.head()

title,#Followme,#Nerealnaya lyubov,$5 a Day,$9.99,$ellebrity,'71,'A' gai wak 2,'Breaker' Morant,'R Xmas,'Round Midnight,...,État de siège,Évolution,Être et avoir,"Ó Paí, Ó",Ôdishon,Ôkami kodomo no Ame to Yuki,Ônibus 174,Ôritsu uchûgun Oneamisu no tsubasa,Üvegtigris,Üç Maymun
title,,,,,,,,,,,,,,,,,,,,,
#Followme,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#Nerealnaya lyubov,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
$5 a Day,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
$9.99,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
$ellebrity,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
recommender.loc['#Followme']

title
#Followme                             1.0
#Nerealnaya lyubov                    0.0
$5 a Day                              0.0
$9.99                                 0.0
$ellebrity                            0.0
                                     ... 
Ôkami kodomo no Ame to Yuki           0.0
Ônibus 174                            0.0
Ôritsu uchûgun Oneamisu no tsubasa    0.0
Üvegtigris                            0.0
Üç Maymun                             0.0
Name: #Followme, Length: 13868, dtype: float64

In [15]:
imdb_titles = pd.read_csv('../Data/Large-Data/imdb_titles.csv')

In [16]:
imdb_titles.head()

,titleId,title
0,tt0000001,Carmencita
1,tt0000002,Le clown et ses chiens
2,tt0000003,Pauvre Pierrot
3,tt0000004,Un bon bock
4,tt0000005,Blacksmith Scene


In [17]:
q = 'Toy Story'

for title in imdb_titles.loc[imdb_titles['title'].str.contains(q), 'title']:
    print(title)
    print(recommender[title].sort_values(ascending = False)[1:11])
    print()
    print()

Toy Story
title
Flicka                                                              0.050089
They Wait                                                           0.050089
DOA: Dead or Alive                                                  0.048198
Fancy Pants                                                         0.047519
Godspell: A Musical Based on the Gospel According to St. Matthew    0.045080
Courageous                                                          0.045080
Talk Radio                                                          0.044801
Ball of Fire                                                        0.043786
Punchline                                                           0.042475
Scooby-Doo and the Loch Ness Monster                                0.040071
Name: Toy Story, dtype: float64


Toy Story 2
title
Nightmare at Noon                                                0.077706
Her Alibi                                                        0.077706
Gertrud       

KeyError: "The Story Behind 'Toy Story'"

## Scale to include more reviews

In [30]:
over_100 = pd.read_csv('../Data/Large-Data/over_100_reviews.csv')
over_100

/var/folders/jc/vvd13n2d3m33dzsrsg9ydhb00000gn/T/ipykernel_2114/3744431260.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  over_100 = pd.read_csv('../Data/Large-Data/over_100_reviews.csv')


,user_id,imdb_id,scaled_rating,title
0,1,tt0110912,10.0,Pulp Fiction
1,1,tt0111495,7.0,Trois couleurs: Rouge
2,1,tt0108394,10.0,Trois couleurs: Bleu
3,1,tt0114787,10.0,Underground
4,1,tt0045152,7.0,Singin' in the Rain
...,...,...,...,...
27522147,ur3174947,tt0096895,8.0,Batman
27522148,ur0581842,tt0107977,6.0,Robin Hood: Men in Tights
27522149,ur3174947,tt0103776,8.0,Batman Returns
27522150,ur4581944,tt0102614,8.0,Out for Justice


In [34]:
print(f"Reviews: {len(over_100['imdb_id'])}")
print(f"Movies: {over_100['imdb_id'].nunique()}")
print(f"Users: {over_100['user_id'].nunique()}")

Reviews: 27522152
Movies: 14339
Users: 1229633


In [35]:
pivot_100 = over_100.pivot_table(values='scaled_rating', index='title', columns='user_id')
pivot_100.head()

/var/folders/jc/vvd13n2d3m33dzsrsg9ydhb00000gn/T/ipykernel_2114/3778825614.py:1: PerformanceWarning: The following operation may generate 16858268430 cells in the resulting pandas object.
  pivot_100 = over_100.pivot_table(values='scaled_rating', index='title', columns='user_id')


: 

In [ ]:
pivot_100 = ml_reviews.pivot_table(values = 'scaled_rating', index = 'title', columns = 'user_id')

pivot_100.head()

In [32]:
pivot_100.shape

In [ ]:
pivot_100_sparse = sparse.csr_matrix(pivot_100.fillna(0))

In [ ]:
distances_100 = pairwise_distances(pivot_100_sparse, metric = 'cosine')

In [ ]:
similarities_100 = 1.0 - distances_100

In [33]:
recommender_100 = pd.DataFrame(similarities_100, index = pivot_100.index, columns=pivot_100.index)

recommender_100.head()

In [ ]:
imdb_titles.head()

,titleId,title
0,tt0000001,Carmencita
1,tt0000002,Le clown et ses chiens
2,tt0000003,Pauvre Pierrot
3,tt0000004,Un bon bock
4,tt0000005,Blacksmith Scene


In [ ]:
q = 'Toy Story'

for title in imdb_titles.loc[imdb_titles['title'].str.contains(q), 'title']:
    print(title)
    print(recommender_100[title].sort_values(ascending = False)[1:11])
    print()
    print()

Toy Story
title
Flicka                                                              0.050089
They Wait                                                           0.050089
DOA: Dead or Alive                                                  0.048198
Fancy Pants                                                         0.047519
Godspell: A Musical Based on the Gospel According to St. Matthew    0.045080
Courageous                                                          0.045080
Talk Radio                                                          0.044801
Ball of Fire                                                        0.043786
Punchline                                                           0.042475
Scooby-Doo and the Loch Ness Monster                                0.040071
Name: Toy Story, dtype: float64


Toy Story 2
title
Nightmare at Noon                                                0.077706
Her Alibi                                                        0.077706
Gertrud       

KeyError: "The Story Behind 'Toy Story'"

# skipping pivot table creation


In [9]:
def create_matrix(df):
     
    N = len(df['user_id'].unique())
    M = len(df['imdb_id'].unique())
     
    # Map Ids to indices
    user_mapper = dict(zip(np.unique(df["user_id"]), list(range(N))))
    movie_mapper = dict(zip(np.unique(df["imdb_id"]), list(range(M))))
     
    # Map indices to IDs
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["user_id"])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["imdb_id"])))
     
    user_index = [user_mapper[i] for i in df['user_id']]
    movie_index = [movie_mapper[i] for i in df['imdb_id']]
 
    X = csr_matrix((df["scaled_rating"], (movie_index, user_index)), shape=(M, N))
     
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper, user_index, movie_index
     


In [10]:
ordered_titles = sampled_reviews.drop_duplicates(subset = 'imdb_id').sort_values(by='imdb_id', ascending=True)['title']

In [13]:
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper, user_index, movie_index = create_matrix(sampled_reviews)

In [14]:
len(movie_mapper)

53786

In [15]:
movie_mapper

{'tt0000001': 0,
 'tt0000003': 1,
 'tt0000007': 2,
 'tt0000008': 3,
 'tt0000010': 4,
 'tt0000012': 5,
 'tt0000013': 6,
 'tt0000014': 7,
 'tt0000016': 8,
 'tt0000022': 9,
 'tt0000023': 10,
 'tt0000027': 11,
 'tt0000028': 12,
 'tt0000029': 13,
 'tt0000033': 14,
 'tt0000041': 15,
 'tt0000070': 16,
 'tt0000075': 17,
 'tt0000091': 18,
 'tt0000131': 19,
 'tt0000132': 20,
 'tt0000138': 21,
 'tt0000192': 22,
 'tt0000211': 23,
 'tt0000230': 24,
 'tt0000242': 25,
 'tt0000246': 26,
 'tt0000274': 27,
 'tt0000299': 28,
 'tt0000300': 29,
 'tt0000304': 30,
 'tt0000313': 31,
 'tt0000355': 32,
 'tt0000358': 33,
 'tt0000359': 34,
 'tt0000364': 35,
 'tt0000399': 36,
 'tt0000410': 37,
 'tt0000417': 38,
 'tt0000420': 39,
 'tt0000423': 40,
 'tt0000437': 41,
 'tt0000439': 42,
 'tt0000447': 43,
 'tt0000455': 44,
 'tt0000465': 45,
 'tt0000469': 46,
 'tt0000488': 47,
 'tt0000498': 48,
 'tt0000499': 49,
 'tt0000516': 50,
 'tt0000524': 51,
 'tt0000534': 52,
 'tt0000546': 53,
 'tt0000553': 54,
 'tt0000554': 55,
 '

In [1]:
#distances = pairwise_distances(X, metric = 'cosine')

NameError: name 'pairwise_distances' is not defined

In [ ]:
#user_index = [user_mapper[i] for i in df['user_id']]
#movie_index = [movie_mapper[i] for i in df['imdb_id']]

##print((df["scaled_rating"], (movie_index, user_index)))

In [16]:
similarities = 1.0 - pairwise_distances(X, metric = 'cosine')

In [ ]:
#similarities = 1.0 - distances

In [17]:
similarities.shape

(53786, 53786)

In [18]:
recommender = pd.DataFrame(similarities, index = ordered_titles, columns=ordered_titles)

recommender.head()

title,Carmencita,Pauvre Pierrot,Corbett and Courtney Before the Kinetograph,Edison Kinetoscopic Record of a Sneeze,La sortie de l'usine Lumière à Lyon,L'arrivée d'un train à La Ciotat,Le débarquement du congrès de photographie à Lyon,L'arroseur arrosé,Barque sortant du port,Les forgerons,...,Fin de siglo,Kaijû no kodomo,Falling Inn Love,Nate Bargatze: The Tennessee Kid,The Far Green Country,Paranormal Investigation,Momenti di trascurabile felicità,Koridor bessmertiya,Lupin the IIIrd: Mine Fujiko no Uso,Kaithi
title,,,,,,,,,,,,,,,,,,,,,
Carmencita,1.000000,0.687353,0.243162,0.221897,0.367293,0.254815,0.630711,0.459319,0.674200,0.667711,...,0.000000,0.0,0.000000,0.000000,0.256323,0.0,0.232391,0.0,0.0,0.0
Pauvre Pierrot,0.687353,1.000000,0.229815,0.199731,0.428713,0.254607,0.575176,0.373734,0.707992,0.696796,...,0.000000,0.0,0.000000,0.000000,0.242253,0.0,0.219635,0.0,0.0,0.0
Corbett and Courtney Before the Kinetograph,0.243162,0.229815,1.000000,0.095388,0.127221,0.240332,0.399556,0.353928,0.360668,0.401846,...,0.000000,0.0,0.000000,0.000000,0.411365,0.0,0.372957,0.0,0.0,0.0
Edison Kinetoscopic Record of a Sneeze,0.221897,0.199731,0.095388,1.000000,0.236683,0.159140,0.347252,0.184956,0.235091,0.234647,...,0.006218,0.0,0.002047,0.002789,0.100551,0.0,0.091163,0.0,0.0,0.0
La sortie de l'usine Lumière à Lyon,0.367293,0.428713,0.127221,0.236683,1.000000,0.407914,0.369062,0.493358,0.372334,0.367538,...,0.000000,0.0,0.000000,0.000000,0.134107,0.0,0.121586,0.0,0.0,0.0


In [19]:
imdb_titles = pd.read_csv('../Data/Large-Data/imdb_titles.csv')

In [20]:
imdb_titles.head()

,titleId,title
0,tt0000001,Carmencita
1,tt0000002,Le clown et ses chiens
2,tt0000003,Pauvre Pierrot
3,tt0000004,Un bon bock
4,tt0000005,Blacksmith Scene


In [21]:
# Get unique IMDb IDs from our movie 
movie_mapper_ids = set(movie_mapper.keys())

# Assuming `imdb_titles` is your DataFrame with columns 'imdb_id' and 'title'
# Filter the DataFrame based on whether the imdb_id exists in movie_mapper_ids
titles_in_movie_mapper = imdb_titles[imdb_titles['titleId'].isin(movie_mapper_ids)]

# Print the resulting DataFrame
titles_in_movie_mapper.head()

,titleId,title
0,tt0000001,Carmencita
2,tt0000003,Pauvre Pierrot
6,tt0000007,Corbett and Courtney Before the Kinetograph
7,tt0000008,Edison Kinetoscopic Record of a Sneeze
9,tt0000010,La sortie de l'usine Lumière à Lyon


In [23]:
x = input('What movie did you just watch?')

In [24]:
x

'Toy Story'

In [22]:
#x = 'Toy Story'

for title in titles_in_movie_mapper.loc[titles_in_movie_mapper['title'].str.contains(x), 'title']:
    print(title)
    print(recommender[title].sort_values(ascending = False)[1:11])
    print()
    print()

Toy Story
title
Star Wars                                         0.564555
Toy Story 2                                       0.564336
Back to the Future                                0.550556
Forrest Gump                                      0.546750
Jurassic Park                                     0.541597
Independence Day                                  0.537916
Star Wars: Episode VI - Return of the Jedi        0.537413
The Lion King                                     0.528811
Aladdin                                           0.525246
Star Wars: Episode V - The Empire Strikes Back    0.513098
Name: Toy Story, dtype: float64


Toy Story 2
title
Toy Story             0.564336
A Bug's Life          0.562676
Monsters, Inc.        0.520420
Shrek                 0.505918
Finding Nemo          0.480358
Ghostbusters          0.470100
Chicken Run           0.467211
Men in Black          0.464324
Back to the Future    0.461655
The Incredibles       0.457856
Name: Toy Story 2, dtype: float6